In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import NetworkAnalysis

import pandas as pd
import numpy as np
import geopandas as gpd
from pydantic import BaseModel
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt
import random

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

import networkx as nx
import ribasim

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
base_dir = Path("C:\\Users\\NLTAND\\OneDrive - Sweco AB\\Algemeen-Tessa\\Projecten\\TKI oppervlaktewatermodule NHI\\")
simulations_dir = Path(base_dir, "d-hydro\\dhydro\\")
date_range = pd.date_range("2000-01-02 23:00", periods=9, freq="2D")

# load areas units (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "wrij_afwateringseenheden_clip_Zutphen\\wrij_afwateringseenheden_clip_Zutphen.shp")
areas = gpd.read_file(areas_file_path)[["geometry"]].to_crs(28992)

# output geopackage
gpkg_path = "output/test_export_geopackage/ribasim_network.gpkg"

In [ ]:
# create networkanalysis
network = NetworkAnalysis(areas_gdf=areas)

# add data from simulation sets
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(simulations_dir, ""),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=date_range,
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(simulations_dir, ""),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=date_range,
)

# get all network data from the schematisations
network.get_network_data()

In [ ]:
# # nodes = network.nodes_gdf.sort_values(by='mesh1d_node_x', ascending=False).iloc[:150]
# nodes = network.nodes_gdf
# nodes_ids = list(nodes.mesh1d_nNodes.values)

# edges = network.edges_gdf
# edges = edges[(edges["start_node_no"].isin(nodes_ids)) & 
#               (edges["end_node_no"].isin(nodes_ids))]

In [ ]:
# Define node_ids on which to split
# split_node_ids = [300, 1200, 1300, 1400, 1430, 1818, 1950, 2380]
# split_node_ids = [2380]
# split_node_ids = list(network.bifurcations_gdf.mesh1d_nNodes.values)
# split_node_ids = list(network.bifurcations_gdf.mesh1d_nNodes.values) + list(network.confluences_gdf.mesh1d_nNodes.values)
split_node_ids = list(network.weirs_gdf.mesh1d_nNodes.values) # all weirs
# split_node_ids = random.sample(split_node_ids, 25)

# create basins (gdf) based on nodes, edges, split_node_ids and areas
basins, areas, nodes, edges = network.create_basins_based_on_split_node_ids(
    # nodes=nodes, 
    # edges=edges, 
    split_node_ids=split_node_ids, 
    # areas=areas
)

In [ ]:
split_nodes = network.nodes_gdf.loc[split_node_ids]
split_nodes['splitnode_id'] = split_nodes['mesh1d_nNodes']
split_nodes = split_nodes[['splitnode_id','geometry']]

In [ ]:
network.export_to_geopackage(basins=basins, split_nodes=split_nodes, output=gpkg_path)

In [ ]:
split_nodes_gdf = gpd.read_file(gpkg_path, layer='split_nodes')

In [ ]:
# create basins (gdf) based on nodes, edges, split_node_ids and areas
basins, areas, nodes, edges = network.create_basins_based_on_split_node_gdf(split_node_gdf=split_nodes_gdf)

In [ ]:
areas.crs

In [ ]:
export_to_geopackage(network = network, basins = basins, split_nodes = split_nodes, output = "output/test_export_geopackage/ribasim_network.gpkg")

In [ ]:
split_nodes

In [ ]:
areas

In [ ]:
areas.loc[42, 'basin'] = 0

In [ ]:
areas

In [ ]:
areas.dissolve(by="basin").reset_index().explode().reset_index(drop=True)